In [4]:
!pip install lpips

import pandas as pd
from skimage.metrics import structural_similarity as ssim
import lpips
import numpy as np
import torch
import skimage.io as io
from scipy.stats import spearmanr
from scipy.io import loadmat

n1=[1,2,3,4]
n2=[5,6,77,9]
spearmanr(n1,n2)[0]

device = 'cuda:2' if torch.cuda.is_available() else 'cpu'
loss_fn_vgg = lpips.LPIPS(net='vgg').to(device)
loss_fn_alex = lpips.LPIPS(net='alex').to(device)
def calculate_lpips(x,y):
    x = torch.tensor(x,device=device)
    y = torch.tensor(y,device=device)
    x = x.permute(2,0,1)
    y = y.permute(2,0,1)
    print(x.shape,y.shape)
    lv=next((loss_fn_vgg(x,y).to('cpu').detach().numpy()).flat)
    la=next((loss_fn_alex(x,y).to('cpu').detach().numpy()).flat)
    return lv,la

device = next(loss_fn_vgg.parameters()).device
print(device)

def calculate_psnr(original, reconstructed):
    mse = np.mean((original - reconstructed) ** 2)
    if mse==0:
        return 100
    max_value = np.max(original)
    psnr = 20 * np.log10(max_value) - 10 * np.log10(mse)
    return psnr

def calculate_ssim(x,y):
    ssim1=ssim(x[:,:,0],y[:,:,0])
    ssim2=ssim(x[:,:,1],y[:,:,1])
    ssim3=ssim(x[:,:,2],y[:,:,2])
    mean=(ssim1+ssim2+ssim3)/3
    return mean

# df = pd.read_csv('/home/tarung/Bharath/AIP/Assignment4/hw5/gblur/gblur/info.txt', delimiter=' ',skiprows=1, header=[0], names=['Image', 'Blurred', 'PSNR'])
# df.drop(' ', axis=1, inplace=True)
df = pd.read_csv('/content/drive/MyDrive/AIP/Assignment_4/hw5/gblur/gblur/info.txt', delimiter=' ', names=['Image', 'Blurred', 'Score',''])
df.drop(df.columns[3], axis=1, inplace=True)
df['PSNR']=None
df['SSIM']=None
df['LPIPS_VGG']=None
df['LPIPS_ALEX']=None
# df1=df.groupby(df.columns[0])
# print(df)
for index, row in df.iterrows():
    ref=io.imread(r'/content/drive/MyDrive/AIP/Assignment_4/hw5/refimgs/refimgs/'+row['Image'])
    img=io.imread(r'/content/drive/MyDrive/AIP/Assignment_4/hw5/gblur/gblur/'+row['Blurred'])
    df.loc[index,'PSNR']=calculate_psnr(ref,img)
    df.loc[index,'SSIM']=calculate_ssim(ref,img)
    print(index)
    df.loc[index,'LPIPS_VGG'],df.loc[index,'LPIPS_ALEX']=calculate_lpips(ref,img)
    # torch.cuda.empty_cache()
    # print(row['Image'],row['Blurred'],calculate_psnr(ref,img))

df

df.to_csv(r'/content/drive/MyDrive/AIP/Assignment_4/Q2_Scores.csv', index=True)

mat=loadmat(r'/content/drive/MyDrive/AIP/Assignment_4/hw5/hw5.mat')
print(mat.keys())

bd=mat['blur_dmos']
# print(bd.shape)
df1=pd.DataFrame()
df1['Spearman_PSNR']=None
df1['Spearman_SSIM']=None
df1['Spearman_LPIPS_VGG']=None
df1['Spearman_LPIPS_ALEX']=None
columns=['PSNR','SSIM','LPIPS_VGG','LPIPS_ALEX']
for col in columns:
    # print(col.shape)
    # print(df[col].shape)
    # print(bd.shape)
    # print(spearmanr(bd.T,df[col])[0])
    df1.loc[0,'Spearman_'+col]=spearmanr(bd.T,df[col])[0]
    # print(df1['Spearman_'+col])
    # df1[col]=spearmanr(bd,df[col])

df1


,Spearman_PSNR,Spearman_SSIM,Spearman_LPIPS_VGG,Spearman_LPIPS_ALEX
0,0.012996,-0.013398,-0.000468,-0.019298
